## Notebook to load exported model from GCP Tables AutoML

### this older post seems most realistic for view the GCP Tables AutoML exported model
ie can't just load model with Keras

http://amygdala.github.io/automl/ml/2019/12/05/automl_tables_export.html

In [1]:
!date

Thu Mar 18 04:08:18 UTC 2021


#### import libraries and set notebook varialbles

In [2]:
import tensorflow as tf
from tensorflow import keras
import sys
# keras.utils.plot_model(model, f'{expr_name}_model_with_shape_info.png', show_shapes=True)

print(tf.version.VERSION)

1.15.4


In [3]:
cohort = 'ppmi'
gene_set = 'elevated'

model_name = 'tf_saved_model-ppmi_elevated_exp_lymphocytes-2020-10-26T01:25:39.798Z'
model_gcs_bckt_path = f'gs://nihnialngcbg-eqtl/amppd/deconvolution/tables_automl/\
model-export/model-export/tbl/{model_name}'
# model_lcl_path = 'model-export'
model_plot_file = f'{cohort}_{gene_set}_model_with_shape_info.png'

#### pull the saved model local

In [4]:
this_cmd = f'gsutil -mq cp -r {model_gcs_bckt_path} .'
print(this_cmd)
!{this_cmd}

gsutil -mq cp -r gs://nihnialngcbg-eqtl/amppd/deconvolution/tables_automl/model-export/model-export/tbl/tf_saved_model-ppmi_elevated_exp_lymphocytes-2020-10-26T01:25:39.798Z .


#### load the model

In [5]:
# expected simple way to load model that won't work
# model = tf.keras.models.load_model(model_name)
# model = tf.saved_model.load(model_name)
# model.summary()

#### bits from the Amy on GCP post for view model using tensorboard after 'convert'

In [6]:
# grab the convert script
!curl -O https://raw.githubusercontent.com/amygdala/code-snippets/master/ml/automl/tables/model_export/convert_oss.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2098  100  2098    0     0  12269      0 --:--:-- --:--:-- --:--:-- 12269


In [13]:
!mkdir converted_export
!python ./convert_oss.py --saved_model {model_name}/saved_model.pb \
--output_dir converted_export

W0317 19:55:41.245392 139940010272576 writer.py:199] Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.
Model Imported. Visualize by running: tensorboard --logdir=converted_export


In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [21]:
%tensorboard --logdir=converted_export

Reusing TensorBoard on port 6006 (pid 25790), started 0:05:23 ago. (Use '!kill 25790' to kill it.)

#### can't the convert works but tensorboard won't ever finish loading the graph
so looking at the convert code, it actually loads the model so try to get summary and figure from that

In [5]:
from absl import app
from absl import flags
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.summary import summary

In [6]:
sm = saved_model_pb2.SavedModel()
graph_def = tf.GraphDef()
with open(f'{model_name}/saved_model.pb', "rb") as pbfile:
    sm.ParseFromString(pbfile.read())
    graph_def.ParseFromString(pbfile.read())
    
print(type(sm))
g_in = tf.import_graph_def(graph_def)
print(type(g_in))

<class 'tensorflow.core.protobuf.saved_model_pb2.SavedModel'>
<class 'NoneType'>


In [7]:
sys.getsizeof(sm)

80

In [8]:
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io

file_content = file_io.read_file_to_string(f'{model_name}/saved_model.pb')
text_format.Merge(file_content.decode("utf-8"), saved_model)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf0 in position 3: invalid continuation byte

In [13]:
# Go through graph and functions to remove every device specification
for mg in sm.meta_graphs:
    for node in mg.graph_def.node:
        node.device = ''
    for func in mg.graph_def.library.function:
        for node in func.node_def:
            node.device = ''
# Write over file
with open(f'{model_name}/saved_model.pb', "wb") as pbfile:    
    pbfile.write(sm.SerializeToString())

#### now try opening new pb file as expected by keras?

In [15]:
tf.contrib.resampler
model = tf.keras.models.load_model(model_name)
model.summary()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



NotFoundError: Op type not registered 'ParseExampleV2' in binary running on gibbsr-automl-model-view. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, 
           show_layer_names=True)